<a href="https://colab.research.google.com/github/curiousNinen/score-prediction/blob/main/classicalStru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import functools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from keras.models import load_model 
from keras.layers import Embedding,Dropout,Conv1D,Dense,GlobalMaxPooling1D,Activation,Flatten,LSTM,BatchNormalization,MaxPooling1D,RNN,GRU
from keras.layers.normalization import BatchNormalization

from keras.callbacks import ModelCheckpoint


def data_import(filepath,classify=True):
    dataset=pd.read_csv(filepath)
    target_score=dataset.pop('Score')
    target_score=target_score.to_frame()
    # target_score=to_categorical(target_score)
    dataset-=np.mean(dataset,axis=0)
    dataset/=np.std(dataset,axis=0)
    if classify:
        target_score= target_score.T
        target_score=tf.one_hot(target_score,6,on_value=1,off_value=-1)
        target_score=tf.reshape(target_score,[dataset.shape[0],-1])
    else:
        target_score=target_score.apply(lambda x: x/100.0)
    return dataset, target_score


filepath_train='/content/drive/My Drive/Colab Notebooks/train2.csv'
filepath_test='/content/drive/My Drive/Colab Notebooks/test2.csv'
filepath_val ='/content/drive/My Drive/Colab Notebooks/validation2.csv'
x_train, y_train= data_import(filepath_train)

# x_train = x_train.drop(x_train.columns[0], axis=1, inplace=True)
# print(type(x_train))
x_test,y_test=data_import(filepath_test)
x_val,y_val = data_import(filepath_val)


x_train=x_train.values.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.values.reshape(x_test.shape[0],x_test.shape[1],1)
x_val=x_val.values.reshape(x_val.shape[0],x_val.shape[1],1)

arg=[128,256,64,6]

class CNNt(tf.keras.Model):
    def __init__(self):
      super(CNNt,self).__init__()
      self.conv1 = Conv1D(arg[0],activation='relu',padding='valid',kernel_size=9)
      self.maxpool1 = MaxPooling1D(pool_size=4, padding='valid')
      self.dense1 = Dense(arg[1],activation='relu',kernel_initializer="uniform")
      self.dense2 = Dense(arg[2],activation='relu',kernel_initializer="uniform")
      self.dense3 = Dense(arg[3],activation='softmax',kernel_initializer="uniform")
      self.flatten = Flatten()
      self.BN = BatchNormalization(axis=1)
    def call(self,x):
      # x=self.drop(x)
      x=self.conv1(x)
      x=self.maxpool1(x)
      # x=self.flatten(x)
      # x=self.BN(x)
      x=self.dense1(x)
      # x=self.drop(x)
      # x=self.activate1(x)
      x=self.flatten(x)
      x=self.BN(x)  
      x=self.dense2(x)
      x=self.dense3(x)
      x=self.flatten(x)
      return x

class LSTMt(tf.keras.Model):
    def __init__(self):
      super(LSTMt,self).__init__()
      # self.denseb = Dense(32,activation='relu')
      # self.lstme = LSTM(128,activation='softsign',return_sequences=True)
      self.lstm1 = LSTM(256,return_sequences=False)
      self.dense0 = Dense(32,activation='relu')
      self.dense = Dense(6,activation='softmax')
      self.BN = BatchNormalization(axis=1)
    def call(self,x):
      # x=self.denseb(x)
      # x=self.lstme(x)
      x=self.lstm1(x)
      x=self.dense0(x)
      x=self.BN(x)
      x=self.dense(x)
      return x

class GRUt(tf.keras.Model):
    def __init__(self):
      super(GRUt,self).__init__()
      self.dense0 = Dense(32,activation='relu')
      self.gru = GRU(256)
      self.dense = Dense(6,activation='softmax')
    def call(self,x):
      x=self.dense0(x)
      x=self.gru(x)
      x=self.dense(x)
      return x
def scheduler(epoch):
    if epoch < 1000:
        return 0.001
    else:
        lr = 0.005 * tf.math.exp(0.1 * (10 - epoch))
        return lr.numpy()
reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)
model=CNNt()

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy',tf.keras.metrics.Precision()])

'''
model.compile(loss='mse',optimizer='adam',metrics=['acc',tf.keras.metrics.Precision()])

# GRU数
model.fit(x_train, y_train,
        batch_size=64,
        epochs=350,
        validation_data=(x_val,y_val))
'''
'''
# LSTM数
model.compile(loss='mse',optimizer='adam',metrics=['acc',tf.keras.metrics.Precision()])
model.fit(x_train, y_train,
        batch_size=128,
        epochs=400,
        validation_data=(x_val,y_val))
'''

# CNN参数
model.compile(loss='mse',optimizer='adam',metrics=['acc',tf.keras.metrics.Precision()])
model.fit(x_train, y_train,
        batch_size=128,
        epochs=400,
        validation_data=(x_val,y_val),
        shuffle=True,
        verbose=2)
# model.fit(x_train, y_train,
#         batch_size=64,
#         epochs=200,
#         shuffle=True,
#         verbose=2,callbacks=[reduce_lr])

print(model.summary())

model.evaluate(x_test,y_test,batch_size=25,verbose=2)
pred=model.predict(x_test)



sns.set()
f,ax=plt.subplots()
predindex = []
for i in range(len(pred)):
  predindex.append(np.argmax(pred[i]))
dataset = pd.read_csv(filepath_test)
y_test = dataset.pop('Score').values
C2= confusion_matrix(y_test, predindex, labels=[0, 1, 2, 3, 4, 5])

np.savetxt('/content/drive/My Drive/Colab Notebooks/matrix.txt',C2)

print(C2) #打印出来看看
sns.heatmap(C2,annot=True,ax=ax, cmap = 'GnBu',center=0.7) #画热力图

ax.set_title('CNNs confusion matrix') #标题
ax.set_xlabel('Predicted label') #x轴
ax.set_ylabel('True lable') #y轴

pred=pd.DataFrame(pred)

# print(predindex)
print(pred)
pred.to_csv('pred_try2.csv')



Epoch 1/400
23/23 - 0s - loss: 1.2140 - acc: 0.3304 - precision_12: 1.0000 - val_loss: 1.2429 - val_acc: 0.3511 - val_precision_12: 0.0000e+00
Epoch 2/400
23/23 - 0s - loss: 1.1632 - acc: 0.4711 - precision_12: 1.0000 - val_loss: 1.2384 - val_acc: 0.3885 - val_precision_12: 0.0000e+00
Epoch 3/400
23/23 - 0s - loss: 1.1153 - acc: 0.5598 - precision_12: 1.0000 - val_loss: 1.2313 - val_acc: 0.4785 - val_precision_12: 1.0000
Epoch 4/400
23/23 - 0s - loss: 1.0666 - acc: 0.6436 - precision_12: 1.0000 - val_loss: 1.2219 - val_acc: 0.5104 - val_precision_12: 1.0000
Epoch 5/400
23/23 - 0s - loss: 1.0278 - acc: 0.7177 - precision_12: 1.0000 - val_loss: 1.2103 - val_acc: 0.6177 - val_precision_12: 1.0000
Epoch 6/400
23/23 - 0s - loss: 1.0002 - acc: 0.7534 - precision_12: 1.0000 - val_loss: 1.1964 - val_acc: 0.6191 - val_precision_12: 1.0000
Epoch 7/400
23/23 - 0s - loss: 0.9788 - acc: 0.7852 - precision_12: 1.0000 - val_loss: 1.1823 - val_acc: 0.6898 - val_precision_12: 1.0000
Epoch 8/400
23/23 -